In [31]:
import psycopg2
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

In [13]:
# Load login data from .env file
load_dotenv()

DATABASE = os.getenv('DB_NAME')
USER_DB = os.getenv('DB_USER')
PASSWORD = os.getenv('DB_PASSWORD')
HOST = os.getenv('DB_HOST')
PORT = os.getenv('DB_PORT')

In [16]:
# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [3]:
# Load data from the database
query_string1 = 'SELECT * FROM "01_bronze"."raw_weather_solar"'
raw_solar = pd.read_sql(query_string1, conn)

query_string2 = 'SELECT * FROM "01_bronze"."raw_weather_wind"'
raw_wind = pd.read_sql(query_string2, conn)

query_string3 = 'SELECT * FROM "01_bronze"."raw_weather_temp"'
raw_temp = pd.read_sql(query_string3, conn)

C:\Users\jakob\AppData\Local\Temp\ipykernel_13584\3282607267.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_solar = pd.read_sql(query_string, conn)
C:\Users\jakob\AppData\Local\Temp\ipykernel_13584\3282607267.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_wind = pd.read_sql(query_string2, conn)
C:\Users\jakob\AppData\Local\Temp\ipykernel_13584\3282607267.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_temp = pd.read_sql(query_string3, conn)


In [24]:
# eor is a column with no value. In the solar data, 'mess_datum' has a minute value of 18, which doesn't match with the other datasets. 
# Instead, we use 'mess_datum_woz' with an adjustment (see below)
raw_wind.drop(['eor'], axis=1, inplace=True)
raw_solar.drop(['eor', 'mess_datum'], axis=1, inplace=True)
raw_temp.drop(['eor'], axis=1, inplace=True)


In [99]:
# Rename columns to something more useful
raw_wind.rename(columns={'stations_id': 'id', 'mess_datum': 'date', 'f': 'w_force', 'd': 'w_direc', 'qn_3': 'qual_w'}, inplace=True)
raw_solar.rename(columns={'stations_id': 'id', 'mess_datum_woz': 'date', 'atmo_lberg': 'atm_rad', 'fd_lberg': 'diff_rad', 
                      'fg_lberg': 'glob_rad', 'sd_lberg': 'sun', 'zenit': 'zenith', 'qn_592': 'qual_s'}, inplace=True)
raw_temp.rename(columns={'stations_id': 'id', 'mess_datum': 'date', 'tt_tu': 'temp', 'rf_tu': 'humid', 'qn_9': 'qual_t'}, inplace=True)

In [100]:
# Replace -999 with NAs
raw_wind.replace(-999, np.nan, inplace=True)
raw_solar.replace(-999, np.nan, inplace=True)
raw_temp.replace(-999, np.nan, inplace=True)

In [101]:
# Convert date to datetime
raw_wind['date'] = pd.to_datetime(raw_wind['date'], format='%Y-%m-%d %H:%M:%S')
raw_solar['date'] = pd.to_datetime(raw_solar['date'], format='%Y%m%d%H:%M')
raw_temp['date'] = pd.to_datetime(raw_temp['date'], format='%Y-%m-%d %H:%M:%S')

In [102]:
# Correct the date in solar by moving it up one hour.
# Example: The original 'mess_datum' might've had 12:18:00 as the time, which 'mess_datum_woz' changed to 13:00:00. 
# But we want that to be rounded down for consistency with the other datasets. 
raw_solar['date'] = raw_solar['date'] - pd.to_timedelta(1, unit='h')

In [103]:
# Two weather stations had entire years worth of data missing, they get dropped
ids_to_drop = ['5779', '5906']
raw_wind = raw_wind[~raw_wind['id'].isin(ids_to_drop)]
raw_solar = raw_solar[~raw_solar['id'].isin(ids_to_drop)]
raw_temp = raw_temp[~raw_temp['id'].isin(ids_to_drop)]

In [104]:
# There are unsystematic missing timeslots throughout the dataset. This code is to create a full date range for each individual weather station, with missing timeslots being filled by NAs
# This is for solar data, the same will be done for wind and temperature.

# Define the complete date range
start_date = raw_solar['date'].min()
end_date = raw_solar['date'].max()
all_hours = pd.date_range(start=start_date, end=end_date, freq='h')

# Create a DataFrame for every combination of id and all_hours
unique_ids = raw_solar['id'].unique()
complete_index = pd.MultiIndex.from_product([unique_ids, all_hours], names=['id', 'date'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge the original data with the complete data
raw_solar_full = pd.merge(complete_df, raw_solar, on=['id', 'date'], how='left')


In [127]:
# Check differences in shape
print(raw_solar.shape)
print(raw_solar_full.shape)

(1980923, 8)
(1980972, 8)


In [74]:
# Check the length of each individual weather station - they're now all equal at the maximum, but one apparently has duplicates.
# This will be checked for the other two datasets as well, where everything is fine. For readability, that has been deleted from the notebook.
pd.crosstab(raw_solar_full['id'], columns='')

col_0,
id,
1048,82536
1358,82536
1684,82536
183,82536
1975,82536
2290,82536
2712,82536
3015,82536
3631,82536


In [105]:
# Eliminate duplicates
df_691 = raw_solar_full[raw_solar_full['id'] == '691']
df_691_unique = df_691.drop_duplicates(subset='date', keep='first')
df_rest = raw_solar_full[raw_solar_full['id'] != '691']
raw_solar_fixed = pd.concat([df_rest, df_691_unique])

In [106]:
# Just to be sure ...
pd.crosstab(raw_solar_fixed['id'], columns='')

col_0,
id,
1048,82536
1358,82536
1684,82536
183,82536
1975,82536
2290,82536
2712,82536
3015,82536
3631,82536


In [128]:
# Check NAs. 49 rows have been added, the other missing values are from the raw data. Lots of data for atmospheric counter-radiation is missing, but lots of missings in the other columns, too
raw_solar_fixed.isna().sum()

id               0
date             0
qual_s          49
atm_rad     996819
diff_rad     41325
glob_rad     15656
sun          11790
zenith          49
dtype: int64

In [129]:
# Some weather stations have more missings than others (global radiation as an example here), but no single one has so many that it would merit dropping it from the dataset (in my opinion)
pd.crosstab(raw_solar_fixed['id'], raw_solar_fixed['glob_rad'].isna())

glob_rad,False,True
id,,
1048,82448,88
1358,80609,1927
1684,79952,2584
183,81994,542
1975,81399,1137
2290,82456,80
2712,82393,143
3015,82527,9
3631,81915,621


In [85]:
# Now we do the same process for temperature. There are values for June 2024 in there, which we don't want for now to keep everything consistent (right?), so we set the end_date specifically

# Define the complete date range
start_date = raw_temp['date'].min()
end_date = pd.Timestamp('2024-05-31 23:00:00')
all_hours = pd.date_range(start=start_date, end=end_date, freq='h')

# Create a DataFrame for every combination of id and all_hours
unique_ids = raw_temp['id'].unique()
complete_index = pd.MultiIndex.from_product([unique_ids, all_hours], names=['id', 'date'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge the original data with the complete data
raw_temp_full = pd.merge(complete_df, raw_temp, on=['id', 'date'], how='left')


In [89]:
raw_temp_full.isna().sum()

id           0
date         0
qual_t     157
temp      3192
humid     3802
dtype: int64

In [130]:
# Again, no major outlier in missings
pd.crosstab(raw_temp_full['id'], raw_temp_full['temp'].isna())

temp,False,True
id,,
1048,82536,0
1358,82365,171
1684,82510,26
183,82267,269
1975,82514,22
2290,82519,17
2712,82456,80
3015,82536,0
3631,82150,386


In [90]:
# Here's wind

# Define the complete date range
start_date = raw_wind['date'].min()
end_date = pd.Timestamp('2024-05-31 23:00:00')
all_hours = pd.date_range(start=start_date, end=end_date, freq='h')

# Create a DataFrame for every combination of id and all_hours
unique_ids = raw_wind['id'].unique()
complete_index = pd.MultiIndex.from_product([unique_ids, all_hours], names=['id', 'date'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge the original data with the complete data
raw_wind_full = pd.merge(complete_df, raw_wind, on=['id', 'date'], how='left')


In [92]:
raw_wind_full.isna().sum()

id             0
date           0
qual_w      7859
w_force    10086
w_direc     8791
dtype: int64

In [95]:
# See above ...
pd.crosstab(raw_wind_full['id'], raw_wind_full['qual_w'].isna(), rownames=['id'], colnames=['NA'])

NA,False,True
id,,
1048,82536,0
1358,81177,1359
1684,82291,245
183,82216,320
1975,82274,262
2290,82498,38
2712,82453,83
3015,82531,5
3631,81844,692


In [108]:
# Export the individual datasets as csv files
raw_wind_full.to_csv('../data/full_weather_wind.csv',index=False)
raw_temp_full.to_csv('../data/full_weather_temp.csv',index=False)
raw_solar_fixed.to_csv('../data/full_weather_solar.csv',index=False)

In [116]:
# Merge the three datasets. Because of the date correction in solar, there's one row for the 31st of december, 2014. I eliminate this here. Maybe there's a better way to do this. 
merged_weather = raw_wind_full.merge(raw_solar_fixed, on=['id', 'date'],how='outer') \
                              .merge(raw_temp_full, on=['id', 'date'], how='outer')

merged_weather = merged_weather.drop(merged_weather.index[0])

In [118]:
# Export the merged dataset as a csv file
merged_weather.to_csv('../data/merged_weather.csv',index=False)